In [6]:
'''spark를 멈추는 메소드'''
# sc.stop()

In [7]:
import pyspark 
from pyspark import SparkContext 
sc = SparkContext()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('yarn').appName('').getOrCreate()       


In [8]:
# 'SparkSession.read.json()'은 dataframe을 반환함
videoJSON = spark.read.json("/example/videodata.json")
# 'SparkSession.sql()'을 통해 sql문을 작성할 때 해당 dataframe 객체를 table로 인식하게 만들기 위해 다음 코드를 입력함
videoJSON.createOrReplaceTempView("videoJSON")

In [10]:
videoJSON

DataFrame[customer_id: bigint, show_id: bigint, state: string, timestamp: bigint]

In [15]:
# 'DataFrame.show()'는 테이블 형식으로 결과물을 출력함
videoJSON.show(100, False)

+-----------+-------+-----------------+----------+
|customer_id|show_id|state            |timestamp |
+-----------+-------+-----------------+----------+
|24         |308    |open             |1510505885|
|78         |580    |open             |1510505888|
|84         |378    |open             |1510505895|
|71         |830    |open             |1510505898|
|64         |473    |open             |1510505903|
|58         |648    |open             |1510505908|
|25         |842    |open             |1510505915|
|67         |628    |open             |1510505919|
|57         |721    |open             |1510505923|
|54         |729    |open             |1510505930|
|38         |107    |open             |1510505933|
|79         |788    |open             |1510505940|
|33         |841    |open             |1510505941|
|46         |566    |open             |1510505946|
|25         |842    |finish_completed |1510505952|
|76         |91     |open             |1510505959|
|90         |707    |open      

In [11]:
videoJSON.count()

1166450

In [17]:
# 'DataFrame.collect()'는 각 행을 요소로 가진 리스트로 결과물을 출력함
spark.sql("select distinct state from videoJSON").collect()

[Row(state='finish_incomplete'),
 Row(state='heartbeat'),
 Row(state='open'),
 Row(state='finish_completed')]

In [20]:
# 'SparkSession.sql()'메소드는 transformation에 해당함. 즉, 새로운 dataframe을 반환함
showid_with_state_open = spark.sql("select show_id, count(*) as cnt_open from videoJSON where state='open' group by show_id")
showid_with_state_open.show()

+-------+--------+
|show_id|cnt_open|
+-------+--------+
|     29|     676|
|     26|     635|
|    964|     666|
|    474|     647|
|    558|     649|
|     65|     694|
|    418|     684|
|    191|     602|
|    541|     643|
|    270|     698|
|    730|     644|
|    293|     641|
|    938|     690|
|    222|     655|
|    720|     720|
|    367|     719|
|    243|     709|
|    442|     687|
|    278|     708|
|    705|     652|
+-------+--------+
only showing top 20 rows



In [21]:
showid_with_state_incomplete = spark.sql("select show_id, count(*) as cnt_Fin from videoJSON where state='finish_incomplete' group by show_id")
showid_with_state_incomplete.show()

+-------+-------+
|show_id|cnt_Fin|
+-------+-------+
|     29|    158|
|     26|    177|
|    964|    166|
|    474|    186|
|    418|    163|
|     65|    190|
|    541|    155|
|    558|    152|
|    191|    139|
|    270|    180|
|    938|    184|
|    730|    160|
|    222|    150|
|    293|    169|
|    720|    173|
|    243|    175|
|    442|    177|
|    705|    146|
|    367|    175|
|    278|    159|
+-------+-------+
only showing top 20 rows



In [23]:
# 'DataFrame.join()'메소드는 transformation에 해당함. 새로운 dataframe을 반환함
left_join = showid_with_state_open.join(showid_with_state_incomplete, showid_with_state_open.show_id == showid_with_state_incomplete.show_id, how = 'left')
left_joi함

DataFrame[show_id: bigint, cnt_open: bigint, show_id: bigint, cnt_Fin: bigint]

In [24]:
left_join.show()

+-------+--------+-------+-------+
|show_id|cnt_open|show_id|cnt_Fin|
+-------+--------+-------+-------+
|     26|     635|     26|    177|
|     29|     676|     29|    158|
|    474|     647|    474|    186|
|    964|     666|    964|    166|
|     65|     694|     65|    190|
|    191|     602|    191|    139|
|    418|     684|    418|    163|
|    541|     643|    541|    155|
|    558|     649|    558|    152|
|    222|     655|    222|    150|
|    270|     698|    270|    180|
|    293|     641|    293|    169|
|    730|     644|    730|    160|
|    938|     690|    938|    184|
|    243|     709|    243|    175|
|    278|     708|    278|    159|
|    367|     719|    367|    175|
|    442|     687|    442|    177|
|    705|     652|    705|    146|
|    720|     720|    720|    173|
+-------+--------+-------+-------+
only showing top 20 rows



In [25]:
import pandas as pd

# 'DataFrame.toPandas()'메소드는 spark dataframe을 pandas dataframe으로 변환시킴
left_join_PD = left_join.toPandas()

In [26]:
left_join_PD['ratio'] = left_join_PD['cnt_Fin'] * 100 / left_join_PD['cnt_open']
left_join_PD

show_id  cnt_open  show_id  cnt_Fin      ratio
0         26       635       26      177  27.874016
1         29       676       29      158  23.372781
2        474       647      474      186  28.748068
3        964       666      964      166  24.924925
4         65       694       65      190  27.377522
..       ...       ...      ...      ...        ...
995      458       651      458      178  27.342550
996      739       705      739      150  21.276596
997      211       671      211      160  23.845007
998      469       642      469      162  25.233645
999      526       694      526      164  23.631124

[1000 rows x 5 columns]